In [1]:
import numpy as np
import pandas as pd

In [39]:
df = pd.read_csv('/content/diabetes.csv')

In [40]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [41]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [42]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [44]:
x = scaler.fit_transform(x)

In [45]:
x[0], x.shape

(array([ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ]), (768, 8))

In [46]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state= 1)

In [95]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [13]:
model = Sequential()
model.add(Dense(32,input_dim= 8 , activation ='relu'))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(optimizer='adam',loss= "binary_crossentropy",metrics=['accuracy'])

In [16]:
model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs = 50)

Epoch 1/50
20/20 [==============================] - 0s 7ms/step - loss: 0.4551 - accuracy: 0.7687 - val_loss: 0.4711 - val_accuracy: 0.7922
Epoch 2/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4541 - accuracy: 0.7736 - val_loss: 0.4722 - val_accuracy: 0.7857
Epoch 3/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4522 - accuracy: 0.7785 - val_loss: 0.4710 - val_accuracy: 0.7922
Epoch 4/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4511 - accuracy: 0.7785 - val_loss: 0.4707 - val_accuracy: 0.7792
Epoch 5/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4501 - accuracy: 0.7752 - val_loss: 0.4700 - val_accuracy: 0.7792
Epoch 6/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4490 - accuracy: 0.7769 - val_loss: 0.4698 - val_accuracy: 0.7792
Epoch 7/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4486 - accuracy: 0.7801 - val_loss: 0.4696 - val_accuracy: 0.7857
Epoch 8/50
20/20 [==

In [23]:
#tuning the otimizer
!pip install -U keras-tuner



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 KB 4.4 MB/s eta 0:00:00


In [48]:
import kerastuner as kt

In [49]:
def b_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

  return model


In [50]:
tuner = kt.RandomSearch(b_model,objective='val_accuracy', max_trials=5)

In [52]:
tuner.search(x_train, y_train, epochs=10,validation_data=(x_test,y_test), verbose=1)

In [54]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [55]:
model = tuner.get_best_models(num_models=1)[0]

In [56]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.fit(x_train,y_train, validation_data=(x_test,y_test), batch_size=32, epochs=100 , initial_epoch=5)

Epoch 6/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5303 - accuracy: 0.7296 - val_loss: 0.5135 - val_accuracy: 0.7792
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5184 - accuracy: 0.7410 - val_loss: 0.5018 - val_accuracy: 0.7662
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5081 - accuracy: 0.7459 - val_loss: 0.4949 - val_accuracy: 0.7662
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4998 - accuracy: 0.7557 - val_loss: 0.4874 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4923 - accuracy: 0.7590 - val_loss: 0.4802 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4865 - accuracy: 0.7606 - val_loss: 0.4739 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4815 - accuracy: 0.7638 - val_loss: 0.4715 - val_accuracy: 0.7857
Epoch 13/

In [64]:
def build_model(hp):
  model= Sequential()

  units = hp.Int("units",min_value=8,max_value=128,step=1)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [65]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='alpha')

In [66]:
tuner.search(x_train,y_train, epochs=5, validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [67]:
tuner.get_best_hyperparameters()[0].values


{'units': 46}

In [68]:
model = tuner.get_best_models(num_models=1)[0]

In [69]:
model.fit(x_train,y_train, validation_data=(x_test,y_test),epochs=100, initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5388 - accuracy: 0.7671 - val_loss: 0.5012 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5143 - accuracy: 0.7704 - val_loss: 0.4831 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4974 - accuracy: 0.7752 - val_loss: 0.4714 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4858 - accuracy: 0.7752 - val_loss: 0.4645 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4778 - accuracy: 0.7736 - val_loss: 0.4575 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4721 - accuracy: 0.7752 - val_loss: 0.4545 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4677 - accuracy: 0.7769 - val_loss: 0.4529 - val_accuracy: 0.7727
Epoch 14

In [70]:
# find No. of layers

def build_model(hp):
  model = Sequential()
  model.add(Dense(46, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    model.add(Dense(72,activation="relu"))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [71]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy', max_trials=5,
                        directory='mydir',
                        project_name='layers_checking')

In [72]:
tuner.search(x_train,y_train, epochs=5, validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 18s


In [73]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [74]:
model = tuner.get_best_models(num_models=1)[0]

In [75]:
model.fit(x_train,y_train, epochs=100, initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 14ms/step - loss: 0.4473 - accuracy: 0.7834 - val_loss: 0.4881 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4382 - accuracy: 0.7883 - val_loss: 0.4857 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4366 - accuracy: 0.7883 - val_loss: 0.4932 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4259 - accuracy: 0.7997 - val_loss: 0.4853 - val_accuracy: 0.8182
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4145 - accuracy: 0.7980 - val_loss: 0.4979 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4057 - accuracy: 0.7997 - val_loss: 0.5015 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4148 - accuracy: 0.8046 - val_loss: 0.4824 - val_accuracy: 0.8117
Epoch 14

In [102]:
def build_model(hp):
  model=Sequential()
  counter = 0
  
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units'+ str(i),min_value=8, max_value=128, step=8 ), 
                      activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid']),
                      input_dim=8
                      ))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+ str(i),min_value=8, max_value=128, step=8 ), 
                      activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid'])
                      ))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      
      counter +=1
  
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','sgd','adam','adadelta','nadam']),
              loss='binary_crossentropy',
              metrics=['accuracy']
  )
  return model

In [103]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='all_in_one_final')

In [104]:
tuner.search(x_train,y_train, epochs = 5 , validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 22s


In [99]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 16,
 'activation0': 'tanh',
 'optimizer': 'sgd',
 'units1': 72,
 'activation1': 'tanh',
 'units2': 104,
 'activation2': 'sigmoid',
 'units3': 112,
 'activation3': 'sigmoid',
 'units4': 120,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'sigmoid',
 'units6': 96,
 'activation6': 'tanh',
 'units7': 64,
 'activation7': 'tanh',
 'units8': 120,
 'activation8': 'relu',
 'units9': 8,
 'activation9': 'tanh'}

In [105]:
model = tuner.get_best_models(num_models=1)[0]

In [106]:
model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=200, initial_epoch=5)

Epoch 6/200
20/20 [==============================] - 2s 16ms/step - loss: 0.6611 - accuracy: 0.6270 - val_loss: 0.6384 - val_accuracy: 0.6429
Epoch 7/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6373 - accuracy: 0.6384 - val_loss: 0.6096 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6254 - accuracy: 0.6433 - val_loss: 0.5858 - val_accuracy: 0.6753
Epoch 9/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6103 - accuracy: 0.6743 - val_loss: 0.5673 - val_accuracy: 0.7078
Epoch 10/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5902 - accuracy: 0.6743 - val_loss: 0.5513 - val_accuracy: 0.7143
Epoch 11/200
20/20 [==============================] - 0s 8ms/step - loss: 0.5797 - accuracy: 0.6824 - val_loss: 0.5396 - val_accuracy: 0.7338
Epoch 12/200
20/20 [==============================] - 0s 8ms/step - loss: 0.5766 - accuracy: 0.6759 - val_loss: 0.5293 - val_accuracy: 0.7597
Epoch 13/